### Import all the necessary libraries

In [48]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch import nn
import matplotlib.pyplot as plt
import requests
import pathlib
import os
import data, data_setup, engine, utils

### Setting up a device agnostic code

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Getting the 20% data for our models

In [50]:
data_20_percent = data.download_data(source_url = 'https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip',
                                     data_folder_name = 'pizza_steak_sushi')

Data\pizza_steak_sushi Already exists, Skipping the creation part...


### Creating training and testing directories

In [51]:
DATA_PATH = pathlib.Path('Data')
IMG_FOLDER = DATA_PATH/'pizza_steak_sushi'

train_dir = IMG_FOLDER/'train'
test_dir = IMG_FOLDER/'test'

### Creating a seed function

In [52]:
def set_seed(seed: int=42):
  torch.cuda.manual_seed(seed)
  torch.manual_seed(seed)

### Getting the pretrained weights for EffNet_b2

In [53]:
! pip install -q torchinfo efficientnet_pytorch

You should consider upgrading via the 'C:\Users\Saud\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [54]:
from torchvision import models
from efficientnet_pytorch import EfficientNet

def effnet_b2_model():
  eff_weights = models.EfficientNet_B2_Weights.DEFAULT
  eff_model = EfficientNet.from_pretrained('efficientnet-b2')
  eff_transform = eff_weights.transforms()
  eff_model.to(device)

  for param in eff_model.parameters():
    param.requires_grad = False

  set_seed(seed=42)
  eff_model._fc = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features= eff_model._fc.in_features, out_features = 3, bias=True)
  ).to(device)

  eff_model.name = 'EfficientNet_b2'
  print(f'[INFO] {eff_model.name} created successfully')
  return eff_model, eff_transform

effb2_model, effb2_transform = effnet_b2_model()

Loaded pretrained weights for efficientnet-b2
[INFO] EfficientNet_b2 created successfully


### Creating datasets, dataloaders and transforms for effnet_b2

In [55]:
BATCH_SIZE = 32
OS_WORKERS = os.cpu_count()


eff_train_dl, eff_test_dl, num_classes, class_idx = data_setup.data_loaders(train_dir=train_dir, test_dir=test_dir,
                                                                    batch_size=BATCH_SIZE, data_transform= effb2_transform,
                                                                            num_os_workers= OS_WORKERS)


### Setting up loss function, optimizer and accuracy function for effnet_b2

In [56]:
def accuracy_score(y_true, y_pred):
  eq = torch.eq(y_true, y_pred).sum().item()
  acc = eq/len(y_pred)
  return acc

eff_loss = nn.CrossEntropyLoss()
eff_optim = torch.optim.Adam(effb2_model.parameters(), lr=1e-3)

### Setting up a training code for effnet_b2 model

In [57]:
# NUM_EPOCHS = 10

# set_seed(seed=42)
# effnetb2_results = engine.train(model = effb2_model, train_dataloader= eff_train_dl,
#                                 test_dataloader = eff_test_dl, loss_fn = eff_loss,
#                                 optimizer = eff_optim, accuracy_fn = accuracy_score, epochs = NUM_EPOCHS,
#                                 device = device)

### Saving effnet_b2 model

In [58]:
MODEL_NAME = 'Effnet_b2_10_epochs_new.pth'
MODEL_PATH = pathlib.Path('Models')
# utils.save_model(model= effb2_model, target_dir = MODEL_PATH, model_name=MODEL_NAME)

### Plotting the loss curves of effnet_b2 model

In [59]:
# import requests

# url = "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py"

# try:
#   from helper_functions import plot_loss_curves
# except:
#   print('[INFO] helper_functions not found, Downloading...')
#   with open('helper_functions.py', 'wb') as f:
#     req = requests.get(url)
#     f.write(req.content)


In [60]:
# from helper_functions import plot_loss_curves

# plot_loss_curves(effnetb2_results)

### Checking the size of effnet_b2 model and other stats

In [61]:
# eff_size = pathlib.Path('Models/Effnet_b2_10_epochs_new.pth').stat().st_size//(1024*1024)
# eff_params = sum(torch.numel(param) for param in effb2_model.parameters())

# stats = {'Test-Loss': effnetb2_results['test_loss'][-1],
#          'Test-Acc': effnetb2_results['test_acc'][-1],
#          'Number of Parameters': eff_params,
#          'Total Size (MB)': eff_size}
# stats

### Getting pretrained weights for ViT_b_16 model

In [62]:
def vit_16_model():
  vit_weights = models.ViT_B_16_Weights.DEFAULT
  vit_model = models.vit_b_16(weights=vit_weights)
  vit_transform = vit_weights.transforms()
  vit_model.to(device)

  for param in vit_model.parameters():
    param.requires_grad = False

  set_seed(seed=42)
  vit_model.heads = nn.Sequential(
      nn.Linear(in_features = 768, out_features = 3)
  ).to(device)

  vit_model.name = 'ViT_b_16_model'
  print(f'[INFO] {vit_model.name} created successfully')
  return vit_model, vit_transform

vitb16_model, vitb16_transform = vit_16_model()

[INFO] ViT_b_16_model created successfully


### Creating datasets, dataloaders and transforms for vit_b_16 model


In [63]:
vit_train_dl, vit_test_dl, num_classes, class_idx = data_setup.data_loaders(train_dir = train_dir, test_dir = test_dir,
                                                                            batch_size = BATCH_SIZE, data_transform = vitb16_transform,
                                                                            num_os_workers=OS_WORKERS)

### Setting up a loss function and optimizer for vit model

In [64]:
vit_loss = nn.CrossEntropyLoss()
vit_optim = torch.optim.Adam(vitb16_model.parameters(), lr=1e-3)

### Setting up a training code for vit model

In [65]:
# set_seed(seed=42)
# vitb16_results = engine.train(model= vitb16_model,
#                               train_dataloader = vit_train_dl,
#                               test_dataloader = vit_test_dl,
#                               epochs= NUM_EPOCHS,
#                               loss_fn = vit_loss,
#                               optimizer= vit_optim,
#                               accuracy_fn = accuracy_score,
#                               device=device)


### Saving the vitb16 model

In [66]:
MODEL2_NAME = 'ViT_b_16_10_epochs_new.pth'

# utils.save_model(model = vitb16_model, target_dir= MODEL_PATH, model_name=MODEL2_NAME)

### Plotting the loss curves for vitb16 model

In [67]:
# from helper_functions import plot_loss_curves

# plot_loss_curves(vitb16_results)

### Checking the size of the vitb16 model and other stats

In [68]:
# vit_params = sum(torch.numel(param) for param in vitb16_model.parameters())
# vit_size = pathlib.Path('Models/ViT_b_16_10_epochs_new.pth').stat().st_size//(1024*1024)

# stats = {'Test-Loss': vitb16_results['test_loss'][-1],
#          'Test-Acc': vitb16_results['test_acc'][-1],
#          'Number of Parameters': vit_params,
#          'Total Size (MB)': vit_size}

# stats

### Creating a path list for test dataset

In [69]:
print(f"[INFO] Displaying the first five image paths out of entire test dataset")
test_data_path = list(pathlib.Path(test_dir).glob('*/*.jpg'))
test_data_path[:5]

[INFO] Displaying the first five image paths out of entire test dataset


[WindowsPath('Data/pizza_steak_sushi/test/pizza/1001116.jpg'),
 WindowsPath('Data/pizza_steak_sushi/test/pizza/1032754.jpg'),
 WindowsPath('Data/pizza_steak_sushi/test/pizza/1067986.jpg'),
 WindowsPath('Data/pizza_steak_sushi/test/pizza/1152100.jpg'),
 WindowsPath('Data/pizza_steak_sushi/test/pizza/129666.jpg')]

### Creating a function for predictions and timing the models


In [70]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict

# 1. Create a function to return a list of dictionaries with sample, truth label, prediction, prediction probability and prediction time
def pred_and_store(paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

    # 2. Create an empty list to store prediction dictionaires
    pred_list = []

    # 3. Loop through target paths
    for path in tqdm(paths):

        # 4. Create empty dictionary to store prediction information for each sample
        pred_dict = {}

        # 5. Get the sample path and ground truth class name
        pred_dict["image_path"] = path
        pred_dict["class_name"] = path.parent.stem

        # 6. Start the prediction timer
        start_time = timer()

        # 7. Open image path
        img = Image.open(path)

        # 8. Transform the image, add batch dimension and put image on target device
        transformed_image = transform(img).unsqueeze(0).to(device)

        # 9. Prepare model for inference by sending it to target device and turning on eval() mode
        model.to(device)
        model.eval()

        # 10. Get prediction probability, predicition label and prediction class
        with torch.inference_mode():
            pred_logit = model(transformed_image) # perform inference on target sample
            pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probabilities
            pred_label = torch.argmax(pred_prob, dim=1) # turn prediction probabilities into prediction label
            pred_class = class_names[pred_label.cpu()] # hardcode prediction class to be on CPU

            # 11. Make sure things in the dictionary are on CPU (required for inspecting predictions later on)
            pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
            pred_dict["pred_class"] = pred_class

            # 12. End the timer and calculate time per pred
            end_time = timer()
            pred_dict["time_for_pred"] = round(end_time-start_time, 4)

        # 13. Does the pred match the true label?
        pred_dict["correct"] = pred_dict['class_name'] == pred_class

        # 14. Add the dictionary to the list of preds
        pred_list.append(pred_dict)

    # 15. Return list of prediction dictionaries
    return pred_list

### Loading the effnet b2 model

In [71]:
effnetb2_loaded = torch.load(MODEL_PATH/MODEL_NAME, map_location= torch.device(type='cpu'))

### Making predictions with effnetb2 model

In [72]:
effb2_model, effb2_transform = effnet_b2_model()

effb2_model.load_state_dict(effnetb2_loaded)

effnetb2_predictions_test = pred_and_store(paths=test_data_path,
                                           model=effb2_model,
                                           transform=effb2_transform,
                                           class_names=num_classes,
                                           device='cpu')


effnetb2_predictions_test[:2]

Loaded pretrained weights for efficientnet-b2
[INFO] EfficientNet_b2 created successfully


100%|██████████| 211/211 [02:37<00:00,  1.34it/s]


[{'image_path': WindowsPath('Data/pizza_steak_sushi/test/pizza/1001116.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9813,
  'pred_class': 'pizza',
  'time_for_pred': 3.1665,
  'correct': True},
 {'image_path': WindowsPath('Data/pizza_steak_sushi/test/pizza/1032754.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.549,
  'pred_class': 'pizza',
  'time_for_pred': 1.9055,
  'correct': True}]

### Loading the vitb16 model

In [73]:
vitb16_loaded = torch.load(MODEL_PATH/MODEL2_NAME, map_location=torch.device(type='cpu'))

### Making predictions with ViTb16 model

In [74]:
vitb16_model, vitb16_transform = vit_16_model()

vitb16_model.load_state_dict(vitb16_loaded)

vitb16_predictions_test = pred_and_store(paths=test_data_path,
                                           model=vitb16_model,
                                           transform=vitb16_transform,
                                           class_names=num_classes,
                                           device='cpu')


vitb16_predictions_test[:2]

[INFO] ViT_b_16_model created successfully


100%|██████████| 211/211 [03:16<00:00,  1.07it/s]


[{'image_path': WindowsPath('Data/pizza_steak_sushi/test/pizza/1001116.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9987,
  'pred_class': 'pizza',
  'time_for_pred': 1.3048,
  'correct': True},
 {'image_path': WindowsPath('Data/pizza_steak_sushi/test/pizza/1032754.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9957,
  'pred_class': 'pizza',
  'time_for_pred': 0.6921,
  'correct': True}]

### Creating dataframe for our ease

In [75]:
import pandas as pd

df1 = pd.DataFrame(effnetb2_predictions_test)
df2 = pd.DataFrame(vitb16_predictions_test)
df1.head()

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,Data\pizza_steak_sushi\test\pizza\1001116.jpg,pizza,0.9813,pizza,3.1665,True
1,Data\pizza_steak_sushi\test\pizza\1032754.jpg,pizza,0.5490,pizza,1.9055,True
2,Data\pizza_steak_sushi\test\pizza\1067986.jpg,pizza,0.9867,pizza,1.5721,True
3,Data\pizza_steak_sushi\test\pizza\1152100.jpg,pizza,0.8414,pizza,1.0168,True
4,Data\pizza_steak_sushi\test\pizza\129666.jpg,pizza,0.9169,pizza,1.0486,True


### Checking for performance (accuracy) and speed (FPS)

In [76]:
df1.correct.value_counts()

correct
True     198
False     13
Name: count, dtype: int64

In [77]:
df2.correct.value_counts()

correct
True     204
False      7
Name: count, dtype: int64

In [78]:
round(df1.time_for_pred.mean(),4)

0.7338

In [79]:
round(df2.time_for_pred.mean(),4)

0.9284

### Creating a gradio demo app

In [80]:
import gradio as gr

In [81]:
next(iter(effb2_model.parameters())).device

device(type='cpu')

### Creating a predict function for our gradio interface

In [82]:
from typing import Tuple, Dict
import random
from PIL import Image
from timeit import default_timer as timer

def predict(img)->Tuple[Dict, float]:

  start_time = timer()

  transformed_img = effb2_transform(img).unsqueeze(dim=0)
  effb2_model.eval()
  with torch.inference_mode():
    logits = effb2_model(transformed_img)
    pred_probs = torch.softmax(logits, dim=1)

  end_time = timer()
  predictions_dict = {num_classes[i]: float(pred_probs[0][i]) for i in range(len(num_classes))}

  pred_time = round(end_time-start_time, 5)
  return predictions_dict, pred_time


### Testing the predict funtion with some random test samples

In [83]:
random_test_sample = random.sample(test_data_path, k=1)[0]
img = Image.open(random_test_sample)
pred_dict, pred_time = predict(img)

print(f'Prediction Class and Probabilities: {pred_dict}')
print(f'Prediction Time: {pred_time}')



Prediction Class and Probabilities: {'pizza': 0.1848863810300827, 'steak': 0.22075796127319336, 'sushi': 0.5943556427955627}
Prediction Time: 2.70941


### Creating a list of lists as required for examples in gradio interface

In [84]:
list_of_examples = [[str(filepath)] for filepath in random.sample(test_data_path, k=3)]
list_of_examples

[['Data\\pizza_steak_sushi\\test\\sushi\\3227791.jpg'],
 ['Data\\pizza_steak_sushi\\test\\steak\\2117351.jpg'],
 ['Data\\pizza_steak_sushi\\test\\steak\\3553838.jpg']]

### Creating Gradio Interface

In [86]:
title = 'FoodVision Mini App'
description = 'An Efficientnetb2 feature extractor computer vision model to classify food images as pizza, steak or sushi'
article = 'Pytorch Model Deployment'
app = gr.Interface(fn = predict,
                   inputs = gr.Image(type='pil'),
                   outputs = [gr.Label(num_top_classes=3, label='Predictions'),
                              gr.Number(label='Predictions Time (s)')],
                   title=title,
                   examples=list_of_examples,
                   description=description,
                   article=article)

app.launch(debug=False, share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a31a867ecfc8a1a779.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
